In [1]:
!pip install sqlalchemy
!pip install setproctitle
!pip install ray[rllib]  # also recommended: ray[debug]
!pip uninstall -y pyarrow

  Created wheel for setproctitle: filename=setproctitle-1.1.10-cp36-cp36m-linux_x86_64.whl size=33936 sha256=88d42ec7b545092f3ef9227bc38473198f9be889238cd54c63ff365537aa5b5f
  Stored in directory: /root/.cache/pip/wheels/e6/b1/a6/9719530228e258eba904501fef99d5d85c80d52bd8f14438a3
Successfully built setproctitle
     |████████████████████████████████| 74.9MB 30.7MB/s 
     |████████████████████████████████| 1.2MB 42.1MB/s 
     |████████████████████████████████| 71kB 24.8MB/s 
     |████████████████████████████████| 389kB 37.2MB/s 
     |████████████████████████████████| 22.1MB 1.2MB/s 
  Found existing installation: protobuf 3.7.1
    Uninstalling protobuf-3.7.1:
      Successfully uninstalled protobuf-3.7.1


Uninstalling pyarrow-0.14.1:
  Successfully uninstalled pyarrow-0.14.1


In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd  /content/drive/My\ Drive/Year\ 4\:\ Synthesis/NS162\:\ Statistical\ Mechanics/Renormalization\ Research\ Project/renormalization/supervised_convnet/t_2.269/3x3/

In [0]:
# %%capture
import train, frozen
import sys
sys.path.insert(0, "../../")
import supervised_convnet
import pickle
from collections import defaultdict
import numpy as np
import time
import torch
# import ray

# # Shut down previous ray
# ray.shutdown()

# # Start Ray.
# ray.init(ignore_reinit_error = True)

mode = "run"
run_mode = "unfrozen_convolution_many_channels"
save_loops = 50
use_cuda = True
results = []
conv_params = {}
filename = ""
def save_progress(results, conv_params, filename, epoch = 500):
    new_results = {}
    new_results["best_val_acc_hist"] = results
    new_results["conv_params"] = conv_params
    new_results["annotation"] = "bias + weights histogram saved"
    new_results["epoch"] = epoch
    # new_results["first_epo    ch_validate_accuracy_list"] = first_epoch_validate_accuracy_list
    with open(filename, "wb") as handle:
        pickle.dump(new_results, handle, protocol = pickle.HIGHEST_PROTOCOL)
"""
Default activation function: sigmoid
"""

# GPU for use in colab
def model_to_cuda(model):
    if use_cuda and torch.cuda.is_available():
        model = model.cuda()
    return model
  
if mode == "run":
    hidden_size = 10
    out_channels = 1
    num_hidden_layers = 1
    conv_params = defaultdict(list)

    if run_mode == "unfrozen_convolution":
        """
        """
        out_channels = 1
        filename = "unfrozen_convolution.pl"
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        for _ in range(500):
            model = supervised_convnet.SupervisedConvNet(filter_size = 3, square_size = 3, \
                    hidden_size = hidden_size, out_channels = out_channels,
                    activation_func = "sigmoid", num_hidden_layers = num_hidden_layers)
            best_val_acc, param_dict, first_epoch_validate_accuracy = train.trainer(model = model)
            results.append(best_val_acc)
            conv_params["weight"].append(param_dict["conv1.weight"])
            conv_params["bias"].append(param_dict["conv1.bias"])
    elif run_mode == "unfrozen_convolution_relu":
        """
        """
        out_channels = 1
        filename = "unfrozen_convolution_relu.pl"
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        results = []
        first_epoch_validate_accuracy_list = []
        for _ in range(500):
            model = supervised_convnet.SupervisedConvNet(filter_size = 3, square_size = 3, \
                    hidden_size = hidden_size, out_channels = out_channels,
                    first_activation = "tanh", activation_func = "relu",
                    num_hidden_layers = num_hidden_layers, seed = time.time() + _)
            model = model_to_cuda(model)
            best_val_acc, param_dict = train.trainer(model = model, batch_size = 86, betas0= 0.0018179320494754046, betas1= 0.001354073715524798,
                                                    lr= 0.004388469485690077, n_epochs= 250, train_size= 4277, weight_decay= 2.346792703571872e-05, use_cuda = use_cuda)
            results.append(best_val_acc)
            conv_params["weight"].append(param_dict["conv1.weight"])
            conv_params["bias"].append(param_dict["conv1.bias"])
            if (_ % save_loops) == (0):
                save_progress(results, conv_params, filename, _)
    elif run_mode == "frozen_convolution_no_center":
        """
        """
        out_channels = 1
        filename = "frozen_convolution_no_center.pl"
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        for _ in range(500):
            model = frozen.SupervisedConvNet(filter_size = 3, square_size = 3, \
                    hidden_size = hidden_size, out_channels = out_channels,
                    center = "omit", activation_func = "sigmoid", num_hidden_layers = num_hidden_layers)
            best_val_acc, param_dict, first_epoch_validate_accuracy = train.trainer(model = model)
            results.append(best_val_acc)
    elif run_mode == "frozen_convolution_no_center_relu":
        """
        """
        filename = "frozen_convolution_no_center_relu.pl"
        out_channels = 1
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        results = []
        for _ in range(500):
            model = frozen.SupervisedConvNet(filter_size = 3, square_size = 3, \
                    hidden_size = hidden_size, out_channels = out_channels,
                    center = "omit", first_activation = "tanh",
                    activation_func = "relu", num_hidden_layers = num_hidden_layers)
            best_val_acc, param_dict = train.trainer(model = model)
            # conv_params["weight"].append(param_dict["conv1.weight"])
            conv_params["bias"].append(param_dict["conv1.bias"])
            results.append(best_val_acc)
    elif run_mode == "frozen_convolution_with_center":
        """
        """
        filename = "frozen_convolution_with_center.pl"
        out_channels = 1
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        for _ in range(500):
            model = frozen.SupervisedConvNet(filter_size = 3, square_size = 3, \
                    hidden_size = hidden_size, out_channels = out_channels,
                    center = "keep", activation_func = "sigmoid", num_hidden_layers = num_hidden_layers)
            best_val_acc, param_dict, first_epoch_validate_accuracy = train.trainer(model = model)
            results.append(best_val_acc)
    elif run_mode == "frozen_convolution_with_center_relu":
        """
        """
        filename = "frozen_convolution_with_center_relu.pl"
        out_channels = 1
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        for _ in range(500):
            model = frozen.SupervisedConvNet(filter_size = 3, square_size = 3, \
                    hidden_size = hidden_size, out_channels = out_channels,
                    center = "keep", activation_func = "relu", num_hidden_layers = num_hidden_layers)
            best_val_acc, param_dict, first_epoch_validate_accuracy = train.trainer(model = model)
            results.append(best_val_acc)
    elif run_mode == "frozen_convolution_pretrained_relu":
        """
        """
        filename = "frozen_convolution_pretrained_relu.pl"
        out_channels = 1
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        results = []
        first_epoch_validate_accuracy_list = []
        for _ in range(500):
            model = frozen.SupervisedConvNet(filter_size = 3, square_size = 3, \
                    hidden_size = hidden_size, out_channels = out_channels,
                    center = "pre_trained", first_activation = "tanh",
                    activation_func = "relu", num_hidden_layers = num_hidden_layers)
            model = model_to_cuda(model)
            best_val_acc, param_dict = train.trainer(model = model, batch_size = 5000, betas0=1- 0.20000000000000004, betas1=1- 1.0081196321129792e-5,
                                                    lr= 0.02564592414390474, n_epochs= 180, train_size= 5000, weight_decay= 6.21073259786956e-05, use_cuda = use_cuda)
            results.append(best_val_acc)
            if (_ % save_loops) == (0):
                save_progress(results, conv_params, filename, _)
    elif run_mode == "unfrozen_convolution_many_channels":
        """
        check to make sure that out_channels below is not 1
        """
        out_channels = 2
        filename = "unfrozen_convolution_{}_channels.pl".format(out_channels)
        print("filename", filename)
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        for _ in range(500):
            model = supervised_convnet.SupervisedConvNet(filter_size = 3, square_size = 3,
                    hidden_size = hidden_size, out_channels = out_channels,
                    activation_func = "sigmoid", num_hidden_layers = num_hidden_layers,
                    seed = time.time() + _)
            model = model_to_cuda(model)
            best_val_acc, param_dict = train.trainer(model = model, batch_size = 5000, betas0=1- 0.20000000000000004, betas1=1- 1.0081196321129792e-5,
                                                    lr= 0.02564592414390474, n_epochs= 180, train_size= 5000, weight_decay= 6.21073259786956e-05, use_cuda = use_cuda)
            results.append(best_val_acc)
            conv_params["weight"].append(param_dict["conv1.weight"])
            conv_params["bias"].append(param_dict["conv1.bias"])
            if (_ % save_loops) == (0):
                save_progress(results, conv_params, filename, _)
    elif run_mode == "unfrozen_convolution_many_channels_relu":
        """
        check to make sure that out_channels below is not 1
        """
        out_channels = 2
        filename = "unfrozen_convolution_{}_channels_relu.pl".format(out_channels)
        print("filename", filename)
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        for _ in range(500):
            model = supervised_convnet.SupervisedConvNet(filter_size = 3, square_size = 3,
                    hidden_size = hidden_size, out_channels = out_channels,
                    first_activation = "tanh", activation_func = "relu",
                    num_hidden_layers = num_hidden_layers, seed = time.time() + _)
            best_val_acc, param_dict = train.trainer(model = model)
            results.append(best_val_acc)
            conv_params["weight"].append(param_dict["conv1.weight"])
            conv_params["bias"].append(param_dict["conv1.bias"])
    elif run_mode == "unfrozen_convolution_many_channel_many_hidden":
        """
        check to make sure that out_channels, num_hidden_layers above is not 1
        """
        out_channels = 2
        num_hidden_layers = 2
        filename = "unfrozen_convolution_{}_channels_{}_hidden.pl".format(out_channels, num_hidden_layers)
        print("filename", filename)
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        for _ in range(500):
            model = supervised_convnet.SupervisedConvNet(filter_size = 3, square_size = 3, \
                    hidden_size = hidden_size, out_channels = out_channels, num_hidden_layers = num_hidden_layers)
            best_val_acc, param_dict, first_epoch_validate_accuracy = train.trainer(model = model)
            results.append(best_val_acc)
            

save_progress(results, conv_params, filename)

    

In [0]:
1# %%capture
import train, frozen
import sys
sys.path.insert(0, "../../")
import supervised_convnet
import pickle
from collections import defaultdict
import numpy as np
import time
import torch
import ray

# Shut down previous ray
ray.shutdown()

# Start Ray.
ray.init(ignore_reinit_error = True)

mode = "run"
run_mode = "frozen_convolution_pretrained_relu"
save_loops = 50
use_cuda = True
results = []
conv_params = {}
filename = ""
def save_progress(results, conv_params, filename, epoch = 500):
    new_results = {}
    new_results["best_val_acc_hist"] = results
    new_results["conv_params"] = conv_params
    new_results["annotation"] = "bias + weights histogram saved"
    new_results["epoch"] = epoch
    # new_results["first_epo    ch_validate_accuracy_list"] = first_epoch_validate_accuracy_list
    with open(filename, "wb") as handle:
        pickle.dump(new_results, handle, protocol = pickle.HIGHEST_PROTOCOL)
"""
Default activation function: sigmoid
"""

@ray.remote(num_return_vals=2)
def init_model_and_train(model, batch_size, betas0, betas1, lr, n_epochs, train_size, weight_decay, use_cuda):
    best_val_acc, param_dict = train.trainer(model = model, batch_size = batch_size, betas0= betas0, betas1= betas1,
                                            lr= lr, n_epochs= n_epochs, train_size= train_size, weight_decay= weight_decay, use_cuda = use_cuda)
    return best_val_acc, param_dict
        

# GPU for use in colab
def model_to_cuda(model):
    if use_cuda and torch.cuda.is_available():
        model = model.cuda()
    return model

if mode == "run":
    hidden_size = 10
    out_channels = 1
    num_hidden_layers = 1
    conv_params = defaultdict(list)

    if run_mode == "unfrozen_convolution":
        """
        """
        out_channels = 1
        filename = "unfrozen_convolution.pl"
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        for _ in range(500):
            model = supervised_convnet.SupervisedConvNet(filter_size = 3, square_size = 3, \
                    hidden_size = hidden_size, out_channels = out_channels,
                    activation_func = "sigmoid", num_hidden_layers = num_hidden_layers)
            best_val_acc, param_dict, first_epoch_validate_accuracy = train.trainer(model = model)
            results.append(best_val_acc)
            conv_params["weight"].append(param_dict["conv1.weight"])
            conv_params["bias"].append(param_dict["conv1.bias"])
    elif run_mode == "unfrozen_convolution_relu":
        """
        """
        out_channels = 1
        filename = "unfrozen_convolution_relu.pl"
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        results = []
        for _ in range(1):
            result_ids = []
            for seed in range(4):
                model = supervised_convnet.SupervisedConvNet(filter_size = 3, square_size = 3, \
                    hidden_size = hidden_size, out_channels = out_channels,
                    first_activation = "tanh", activation_func = "relu",
                    num_hidden_layers = num_hidden_layers, seed = time.time() + seed)
                model = model_to_cuda(model)
                result_ids.append(init_model_and_train.remote(model = model, batch_size = 86, betas0=1- 0.0018179320494754046, betas1= 1-0.001354073715524798,
                                                    lr= 0.004388469485690077, n_epochs= 250, train_size= 4277, weight_decay= 2.346792703571872e-05, use_cuda = use_cuda))
            
            # Wait for the tasks to complete and retrieve the results.
            # With at least 4 cores, this will take 1 second.
            results.append(ray.get(result_ids))  # [0, 1, 2, 3]
            
            best_val_acc, param_dictionaries = zip(*results)
            results.append(best_val_acc)
            for param_dict in param_dictionaries:
                conv_params["weight"].append(param_dict["conv1.weight"])
                conv_params["bias"].append(param_dict["conv1.bias"])
            print("best_val_acc", best_val_acc)
            print("conv_params", conv_params)
            if (_ % save_loops) == (0):
                save_progress(best_val_acc, conv_params, filename, _)
    elif run_mode == "frozen_convolution_no_center":
        """
        """
        out_channels = 1
        filename = "frozen_convolution_no_center.pl"
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        for _ in range(500):
            model = frozen.SupervisedConvNet(filter_size = 3, square_size = 3, \
                    hidden_size = hidden_size, out_channels = out_channels,
                    center = "omit", activation_func = "sigmoid", num_hidden_layers = num_hidden_layers)
            best_val_acc, param_dict, first_epoch_validate_accuracy = train.trainer(model = model)
            results.append(best_val_acc)
    elif run_mode == "frozen_convolution_no_center_relu":
        """
        """
        filename = "frozen_convolution_no_center_relu.pl"
        out_channels = 1
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        results = []
        for _ in range(500):
            model = frozen.SupervisedConvNet(filter_size = 3, square_size = 3, \
                    hidden_size = hidden_size, out_channels = out_channels,
                    center = "omit", first_activation = "tanh",
                    activation_func = "relu", num_hidden_layers = num_hidden_layers)
            best_val_acc, param_dict = train.trainer(model = model)
            # conv_params["weight"].append(param_dict["conv1.weight"])
            conv_params["bias"].append(param_dict["conv1.bias"])
            results.append(best_val_acc)
    elif run_mode == "frozen_convolution_with_center":
        """
        """
        filename = "frozen_convolution_with_center.pl"
        out_channels = 1
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        for _ in range(500):
            model = frozen.SupervisedConvNet(filter_size = 3, square_size = 3, \
                    hidden_size = hidden_size, out_channels = out_channels,
                    center = "keep", activation_func = "sigmoid", num_hidden_layers = num_hidden_layers)
            best_val_acc, param_dict, first_epoch_validate_accuracy = train.trainer(model = model)
            results.append(best_val_acc)
    elif run_mode == "frozen_convolution_with_center_relu":
        """
        """
        filename = "frozen_convolution_with_center_relu.pl"
        out_channels = 1
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        for _ in range(500):
            model = frozen.SupervisedConvNet(filter_size = 3, square_size = 3, \
                    hidden_size = hidden_size, out_channels = out_channels,
                    center = "keep", activation_func = "relu", num_hidden_layers = num_hidden_layers)
            best_val_acc, param_dict, first_epoch_validate_accuracy = train.trainer(model = model)
            results.append(best_val_acc)
    elif run_mode == "frozen_convolution_pretrained_relu":
        """
        """
        filename = "frozen_convolution_pretrained_relu.pl"
        out_channels = 1
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        results = []
        for _ in range(1):
            result_ids = []
            for seed in range(4):
                model = frozen.SupervisedConvNet(filter_size = 3, square_size = 3, \
                    hidden_size = hidden_size, out_channels = out_channels,
                    center = "pre_trained", first_activation = "tanh",
                    activation_func = "relu", num_hidden_layers = num_hidden_layers)
                model = model_to_cuda(model)
                result_ids.append(init_model_and_train.remote(model = model, batch_size = 5000, betas0=1- 0.20000000000000004, betas1= 1- 1.0081196321129792e-5,
                                                    lr= 0.02564592414390474, n_epochs= 180, train_size= 5000, weight_decay= 6.21073259786956e-05, use_cuda = use_cuda))
            
            # Wait for the tasks to complete and retrieve the results.
            # With at least 4 cores, this will take 1 second.
            results.append(ray.get(result_ids))  # [0, 1, 2, 3]
            
            best_val_acc, param_dictionaries = zip(*results)
            results.append(best_val_acc)
            for param_dict in param_dictionaries:
                conv_params["weight"].append(param_dict["conv1.weight"])
                conv_params["bias"].append(param_dict["conv1.bias"])
            print("best_val_acc", best_val_acc)
            print("conv_params", conv_params)
            if (_ % save_loops) == (0):
                save_progress(best_val_acc, conv_params, filename, _)
                
    elif run_mode == "unfrozen_convolution_many_channels":
        """
        check to make sure that out_channels below is not 1
        """
        out_channels = 2
        filename = "unfrozen_convolution_{}_channels.pl".format(out_channels)
        print("filename", filename)
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        for _ in range(500):
            model = supervised_convnet.SupervisedConvNet(filter_size = 3, square_size = 3,
                    hidden_size = hidden_size, out_channels = out_channels,
                    activation_func = "sigmoid", num_hidden_layers = num_hidden_layers,
                    seed = time.time() + _)
            best_val_acc, param_dict, first_epoch_validate_accuracy = train.trainer(model = model)
            results.append(best_val_acc)
            conv_params["weight"].append(param_dict["conv1.weight"])
            conv_params["bias"].append(param_dict["conv1.bias"])
    elif run_mode == "unfrozen_convolution_many_channels_relu":
        """
        check to make sure that out_channels below is not 1
        """
        out_channels = 2
        filename = "unfrozen_convolution_{}_channels_relu.pl".format(out_channels)
        print("filename", filename)
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        for _ in range(500):
            model = supervised_convnet.SupervisedConvNet(filter_size = 3, square_size = 3,
                    hidden_size = hidden_size, out_channels = out_channels,
                    first_activation = "tanh", activation_func = "relu",
                    num_hidden_layers = num_hidden_layers, seed = time.time() + _)
            best_val_acc, param_dict = train.trainer(model = model)
            results.append(best_val_acc)
            conv_params["weight"].append(param_dict["conv1.weight"])
            conv_params["bias"].append(param_dict["conv1.bias"])
    elif run_mode == "unfrozen_convolution_many_channel_many_hidden":
        """
        check to make sure that out_channels, num_hidden_layers above is not 1
        """
        out_channels = 2
        num_hidden_layers = 2
        filename = "unfrozen_convolution_{}_channels_{}_hidden.pl".format(out_channels, num_hidden_layers)
        print("filename", filename)
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        for _ in range(500):
            model = supervised_convnet.SupervisedConvNet(filter_size = 3, square_size = 3, \
                    hidden_size = hidden_size, out_channels = out_channels, num_hidden_layers = num_hidden_layers)
            best_val_acc, param_dict, first_epoch_validate_accuracy = train.trainer(model = model)
            results.append(best_val_acc)
            

save_progress(results, conv_params, filename)

    

In [0]:
i = 0